In [1]:
type Or[A, B] = Either[A, B]
type And[A, B] = (A, B)
type Not[A] = A => Nothing



defined type Or
defined type And
defined type Not

In [2]:
trait CL[L[_]]{    

    def lam[A, B](f: L[A] => L[B]): L[A => B]
    def app[A, B](f: L[A => B], a: L[A]): L[B]
    
    def not[A](f: L[A] => L[Nothing]): L[Not[A]] = 
        lam[A, Nothing](f)
    
    def elimNothing[A](ab: L[Nothing]): L[A]
    def elimNot[A, B](n: L[Not[A]], a: L[A]): L[B]
    
    def and[A, B](a: L[A], b: L[B]): L[(A, B)]
    def elimAndL[A, B](a: L[(A, B)]): L[A]
    def elimAndR[A, B](a: L[(A, B)]): L[B]
    
    def orL[A, B](a: L[A]): L[A Either B]
    def orR[A, B](b: L[B]): L[A Either B]
    def elimOr[A, B, C](
        or: L[A Either B], 
        fa: L[A => C],
        fb: L[B => C]): L[C]
    
}

object CL{
    object Syntax{
        implicit class Ops[A, B, L[_]](f: L[A => B])(implicit L: CL[L]){
            def apply(a: L[A]): L[B] = L.app(f, a)
        }

        implicit def toLam[L[_], A, B](f: L[A] => L[B])(implicit L: CL[L]): L[A => B] = 
            L.lam(f)
    }
    
    type Id[T] = T 
    
    object IdCL extends CL[Id]{
        def lam[A, B](f: A => B): A => B = f
        def app[A, B](f: A => B, a: A): B = f(a)

        def elimNothing[A](ab: Nothing): A = ab
        def elimNot[A, B](n: Not[A], a: A): B = n(a)


        def and[A, B](a: A, b: B): (A, B) = (a, b)
        def elimAndL[A, B](a: (A, B)): A = a._1
        def elimAndR[A, B](a: (A, B)): B = a._2

        def orL[A, B](a: A): A Either B = Left(a)
        def orR[A, B](b: B): A Either B = Right(b)
        def elimOr[A, B, C](
            or: A Either B, 
            fa: A => C,
            fb: B => C): C = or.fold(fa, fb)
    }
}

defined trait CL
defined object CL

In [3]:
abstract class Proofs[L[_]]{
    implicit val L: CL[L]
    import CL.Syntax._
    import L._
    
    def proof1[A, B]: L[(A Or B) => Not[A] => B] = 
        lam((or: L[A Or B]) => 
            lam((n: L[Not[A]]) => 
                elimOr(or,
                    lam((a: L[A]) => elimNot[A, B](n, a)), 
                    lam((b: L[B]) => b))))

    def proof2[A, B]: L[Not[A] => A => B] = 
        lam(n => 
            lam((a: L[A]) => 
                elimNot[A, B](n, a)))

    def proof3[A, B]: L[(A => Not[B]) => (A => B) => Not[A]] = 
        lam((i: L[A => Not[B]]) => 
            lam((i2: L[A => B]) => 
                not((a: L[A]) => elimNot(i(a), i2(a)))))
/*
    def proof4[A, B]: Not[A => B] => Not[B] = 
        n => (b: B) => n((_: A) => b)

    def proof5[A, B]: Not[A] => Not[And[A, B]] = 
    //    (and: And[A, B]) => n(And.elimL(and))
        n => n compose And.elimL

    def proof6[A, B]: Not[And[A, Not[A]]] = 
        (a: And[A, Not[A]]) => And.elimR(a)(And.elimL(a))

    def proof7[A, B]: Not[And[A, B]] => A => Not[B] = 
        n => (a: A) => (b: B) => n(And(a, b))

    def proof8[A, B]: Or[A, B] => Or[B, A] = 
        Or.elim[A, B, Or[B, A]](Or.applyR[B, A], Or.applyL[B, A])

    def proof9[A, B]: Not[A Or B] => Not[A] And Not[B] = 
        n => And((a: A) => n(Or.applyL(a)), (b: B) => n(Or.applyR(b)))

    def proof10[A, B]: (Not[A] And Not[B]) => Not[A Or B] = 
        and => Or.elim[A, B, Absurd](And.elimL(and), And.elimR(and))

    def proof11[A]: Not[A And Not[A]] = 
        a => And.elimR(a)(And.elimL(a))
        */
}

defined class Proofs

In [6]:
// derivation rules

sealed abstract class Derivation[E, A]

case class VarZ[A, E]() 
extends Derivation[(A, E), A]

case class VarS[A, B, E](
    d: Derivation[E, A]
) extends Derivation[(B, E), A]

case class ImplI[A, B, E](
    d1: Derivation[(A, E), B]
) extends Derivation[E, A => B]

case class ImplE[A, B, E](
    d1: Derivation[E, A => B],
    d2: Derivation[E, A]
) extends Derivation[E, B]

case class NotE[A, B, E](
    d1: Derivation[E, A],
    d2: Derivation[E, Not[A]]
) extends Derivation[E, B]

case class AndI[A, B, E](
    d1: Derivation[E, A],
    d2: Derivation[E, B]
) extends Derivation[E, A And B]

case class AndE1[A, B, E](
    d1: Derivation[E, A And B]
) extends Derivation[E, A]

case class AndE2[A, B, E](
    d1: Derivation[E, A And B]
) extends Derivation[E, B]

case class OrI1[A, B, E](
    d1: Derivation[E, A]
) extends Derivation[E, A Or B]

case class OrI2[A, B, E](
    d1: Derivation[E, B]
) extends Derivation[E, A Or B]

case class OrE[A, B, C, E](
    d1: Derivation[E, A Or B],
    d2: Derivation[E, A => C],
    d3: Derivation[E, B => C]
) extends Derivation[E, C]


defined class Derivation
defined class VarZ
defined class VarS
defined class ImplI
defined class ImplE
defined class NotE
defined class AndI
defined class AndE1
defined class AndE2
defined class OrI1
defined class OrI2
defined class OrE

In [7]:
sealed abstract class Ctx[E]
case class CtxZ() extends Ctx[Unit]
case class CtxS[A, E](c: Ctx[E]) extends Ctx[(A, E)]

trait TBD[A]{
    def apply[E](c: Ctx[E]): Derivation[E, A]
}

object TBD{
    
    implicit object TBD_CL extends CL[TBD]{
        /*
        def lam[A, B](f: L[A] => L[B]): L[A => B]
*/
        def app[A, B](f: TBD[A => B], a: TBD[A]): TBD[B] = 
            new TBD[B]{
                def apply[E](c: Ctx[E]): Derivation[E, B] = 
                    ImplE(f(c), a(c))
            }
/*
        def not[A](f: L[A] => L[Nothing]): L[Not[A]] = 
            lam[A, Nothing](f)

        def elimNothing[A](ab: L[Nothing]): L[A]
        def elimNot[A, B](n: L[Not[A]], a: L[A]): L[B]
*/
        def and[A, B](a: TBD[A], b: TBD[B]): TBD[(A, B)] = 
            new TBD[(A, B)]{
                def apply[E](c: Ctx[E]): Derivation[E, (A, B)] = 
                    AndI(a(c), b(c))
            }
        
        /*
        def elimAndL[A, B](a: L[(A, B)]): L[A]
        def elimAndR[A, B](a: L[(A, B)]): L[B]

        def orL[A, B](a: L[A]): L[A Either B]
        def orR[A, B](b: L[B]): L[A Either B]
        def elimOr[A, B, C](
            or: L[A Either B], 
            fa: L[A => C],
            fb: L[B => C]): L[C]
    */
    }
}

cmd7.sc:11: object creation impossible.
Missing implementations for 8 members. Stub implementations follow:
  def elimAndL[A, B](a: Helper.this.TBD[(A, B)]): Helper.this.TBD[A] = ???
  def elimAndR[A, B](a: Helper.this.TBD[(A, B)]): Helper.this.TBD[B] = ???
  def elimNot[A, B](n: Helper.this.TBD[ammonite.$sess.cmd1.wrapper.cmd0.Not[A]], a: Helper.this.TBD[A]): Helper.this.TBD[B] = ???
  def elimNothing[A](ab: Helper.this.TBD[Nothing]): Helper.this.TBD[A] = ???
  def elimOr[A, B, C](or: Helper.this.TBD[Either[A,B]], fa: Helper.this.TBD[A => C], fb: Helper.this.TBD[B => C]): Helper.this.TBD[C] = ???
  def lam[A, B](f: Helper.this.TBD[A] => Helper.this.TBD[B]): Helper.this.TBD[A => B] = ???
  def orL[A, B](a: Helper.this.TBD[A]): Helper.this.TBD[Either[A,B]] = ???
  def orR[A, B](b: Helper.this.TBD[B]): Helper.this.TBD[Either[A,B]] = ???

    implicit object TBD_CL extends CL[TBD]{
                    ^Compilation Failed

: 